In [14]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import OperationalError
import json
import re
import config

In [15]:
job_types = {
    'normal':{"data-automation": "normalJob"},
    'sponsored':{"data-automation": "premiumJob"}
}

attributes ={
    'job_name': {'data-automation': 'jobTitle'},
    'company': {'data-automation': 'jobCompany'},
    'location': {'data-automation': 'jobLocation'},
    'pay': {'data-automation': 'jobSalary'},
    'category': {'data-automation': 'jobClassification'},
    'sub_category': {'data-automation': 'jobSubClassification'},
    'job_link': {'data-automation': 'jobSubClassification'}
}

daterange=2

In [16]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import config
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import OperationalError
import json
import re

def sleeper(min_second=0.2, max_second=2):
    """

    :param min_second: minimum seconds
    :param max_second: maximum seconds
    :return: sleep for random period between min and max seconds

    :note: Need to update this to have proper handling for the seconds
    """
    multiplier = 10
    min_time = min_second * multiplier
    max_time = max_second * multiplier
    time.sleep(random.randint(min_time, max_time) / multiplier)


def db_connection():
    with open('../1. Admin/db_data.json') as json_data:
        db_data = json.load(json_data)
        conn = create_connection(
            db_data['db_name'], db_data['db_user'], db_data['db_password'], db_data['db_host'], db_data['db_port']
        )

    return conn


def job_info_handler(job_soup):
    """
    :param job_soup: The html of the jobs - This is a list of the html for each job. Each entry is one job.
    :return: Return a list of jobs in dictionaries. Details of the jobs are contained in the dictionaries.
    """
    job_output = []
    for j in job_soup:

    #Create the dictionary that gets output for each job. Specific handling for each item is in the config file.
        job_info = {
            'job_type': '',
            "job_name": "",
            "company": "",
            "location": "",
            "pay": "",
            "category": "",
            'sub_category': '',
            "first_seen": datetime.datetime.now(),
            "job_link": "",
            "site_id": ""
        }

        for k, v in config.attributes.items():
            var = j.find(attrs=v)
            if var:
                job_info[k] = var.getText()

            # This is terrible, look for a solution
            job_info['job_link'] = j.find(attrs={'data-automation': 'jobTitle'})['href']
            job_info['site_id'] = re.search('(\d+)', job_info['job_link']).group(0)

        job_output.append(job_info)

    return job_output


def search_site(site_link):
    """
    :param site_link: The site link that is to be searched - Currently just a an entry in config, will need to consider how to develop further.
    :return: Returns all jobs from the page after looping through all pages.
    """
    page_number = 1
    output_list = []
    start_flag = True
    while start_flag:
        link = site_link + str(page_number)
        page = requests.get(link)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        norm_jobs = html_soup.find_all("article", {"data-automation": "normalJob"})
        prem_job = html_soup.find_all("article", {"data-automation": "premiumJob"})
        norm_jobs.extend(prem_job)

        if not norm_jobs:
            break

        output_list.extend(job_info_handler(norm_jobs))
        sleeper()
        page_number += 1

    return output_list


def remove_previous_entries(job_ad_list):
    """
    Moved this out of the 'search site' function as there were times where we were dropping all records and it was causing issues
    :param job_ad_list: List of job ads from the 'search site' function
    :return: Same list but with all entries already in db removed
    """
    existing_id = pd.read_sql_query(config.existing_id_sql, con=db_connection())['site_id'].tolist()
    job_ad_list[:] = [d for d in job_ad_list if d.get('site_id') not in existing_id]

    return job_ad_list


def search_job_ad_details(job_info_list):
    """
    :param job_info_list: Cleaned list of jobs broken into individual dictionaries
    :return: Returns the cleaned list of jobs with the job description added to each entry.
    """
    for x in job_info_list:
        base_link = 'https://www.seek.com.au'
        job_link = base_link + x['job_link']
        job_page = requests.get(job_link)
        job_page_soup = BeautifulSoup(job_page.text, 'html.parser')
        x['job_ad_details'] = job_page_soup.find(attrs={'data-automation': 'jobAdDetails'}).getText()
        sleeper()

    print('Job Details Completed')


def create_connection(db_name, db_user, db_password, db_host, db_port):
    """

    :param db_name: database name
    :param db_user: username
    :param db_password: password
    :param db_host: host location
    :param db_port: port to access
    :return: Gives us the connection to db
    """
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection


def execute_many(conn, df, table):
    """
    This is an insert command, so the table needs to be created already.
    :param conn: pre-created connection to db
    :param df: df to insert
    :param table: table to insert df into
    :return: insert records into a table
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()


def seek_process():
    """
    Complete seek process, scraping details and saving these into the pre-defined table
    :return: update db

    :note: Need to improve this with logging and messages. Also update the connection details to work off a json before we save to git
    """

    print('Search Site')
    results = []
    for job_classification in config.classification_search_list:
        seek_link = f'https://www.seek.com.au/{job_classification}?daterange={config.daterange}&page='
        print(job_classification)
        site_results = search_site(seek_link)
        print(len(site_results))
        results.extend(site_results)
    results = remove_previous_entries(results)
    print(f'Getting job details for {len(results)} jobs')
    search_job_ad_details(results)
    df = pd.DataFrame(results)
    df = df.replace(r'^\s*$', np.nan, regex=True)
    print(f'Writing data to db - {df.shape[0]} rows')

    with open('../1. Admin/db_data.json') as json_data:
        db_data = json.load(json_data)
        conn = create_connection(
            db_data['db_name'], db_data['db_user'], db_data['db_password'], db_data['db_host'], db_data['db_port']
        )

    execute_many(conn, df, 'jobs_details')
    


In [65]:
output_list=[]
link =  f'https://www.seek.com.au/jobs?daterange={daterange}&page=1'
page = requests.get(link)
html_soup = BeautifulSoup(page.text, 'html.parser')
norm_jobs = html_soup.find_all("article", {"data-automation": "normalJob"})
prem_job = html_soup.find_all("article", {"data-automation": "premiumJob"})
norm_jobs.extend(prem_job)

output_list.extend(job_info_handler(norm_jobs))

In [16]:
with open('../1. Admin/db_data.json') as json_data:
    db_data = json.load(json_data)
    conn = create_connection(
        db_data['db_name'], db_data['db_user'], db_data['db_password'], db_data['db_host'], db_data['db_port']
    )

Connection to PostgreSQL DB successful


In [71]:
existing_id=pd.read_sql_query(f'select site_id from jobs_details where first_seen >= current_date-{daterange}',con=conn)['site_id'].tolist()

In [72]:
existing_id

['67672445',
 '67678974',
 '67659775',
 '67655271',
 '67657549',
 '67659207',
 '67641989',
 '67642445',
 '67637889',
 '67628402',
 '67626405',
 '67639692',
 '67639227',
 '67629789',
 '67631097',
 '67641586',
 '67637517',
 '67645855',
 '67626470',
 '67626159',
 '67646379',
 '67639267',
 '67625709',
 '67644104',
 '67629074',
 '67639392',
 '67630443',
 '67637335',
 '67644154',
 '67635421',
 '67640893',
 '67643758',
 '67631171',
 '67633708',
 '67628925',
 '67639970',
 '67641453',
 '67626742',
 '67643928',
 '67641295',
 '67626787',
 '67640662',
 '67628046',
 '67636636',
 '67633462',
 '67631744',
 '67642125',
 '67641823',
 '67626243',
 '67626146',
 '67625512',
 '67627961',
 '67641492',
 '67627901',
 '67625978',
 '67637836',
 '67626398',
 '67632686',
 '67635762',
 '67628588',
 '67667122',
 '67641295',
 '67640580',
 '67639814',
 '67640250',
 '67628516',
 '67635586',
 '67635229',
 '67625843',
 '67631830',
 '67625954',
 '67625630',
 '67634472',
 '67636073',
 '67637055',
 '67639144',
 '67637159',

In [40]:
for j in norm_jobs:
    for k, v in attributes.items():
        var = j.find(attrs=v)
        if var:
            k = var.getText()

In [46]:
re.search('(\d+)', k)

In [66]:
len(output_list)

22

In [68]:
output_list[:] = [d for d in output_list if d.get('site_id') not in existing_id]

In [69]:
len(output_list)

17

In [70]:
output_list

[{'job_type': '',
  'job_name': 'Virtual Assistant |\xa0Work From Home |\xa0School Hours (NSW)',
  'company': 'Offsiters',
  'location': 'Sydney NSW',
  'pay': '$30 – $35 per hour',
  'category': '(Administration & Office Support)',
  'sub_category': 'Administrative Assistants',
  'first_seen': datetime.datetime(2023, 5, 24, 20, 46, 49, 935400),
  'job_link': '/job/67697531?type=standout',
  'site_id': '67697531'},
 {'job_type': '',
  'job_name': 'Virtual Assistant |\xa0Work From Home |\xa0School Hours (VIC)',
  'company': 'Offsiters',
  'location': 'Melbourne VIC',
  'pay': '$30 – $35 per hour',
  'category': '(Administration & Office Support)',
  'sub_category': 'Administrative Assistants',
  'first_seen': datetime.datetime(2023, 5, 24, 20, 46, 49, 936400),
  'job_link': '/job/67697585?type=standout',
  'site_id': '67697585'},
 {'job_type': '',
  'job_name': 'Customer Service Officer - Hybrid Work From Home',
  'company': 'Plan Partners',
  'location': 'Brisbane QLD',
  'pay': '',
  

Testing Append

In [1]:
results = []
test_out = ['Hello', 'Yes']
results.extend(test_out)

In [2]:
results

['Hello', 'Yes']

In [3]:
results = []
test_out = []
results.extend(test_out)

In [9]:
results = []
test_out = ['Hello', 'Yes']
results.extend(test_out)
results

['Hello', 'Yes']

In [17]:
out = requests.get('https://www.seek.com.au/job/67729590')

In [20]:
out.history

[]

In [21]:
r = requests.get('http://github.com/')

In [22]:
r.url

'https://github.com/'

In [24]:
r.status_code

200

In [25]:
r.history

[<Response [301]>]

In [26]:
requests.codes.ok

200

Test Dictionary

In [67]:
test_dict = [{'id': 1,
             'name': 'Jack'},
            {'id': 2,
            'name': 'Alex'},
            {'broken':'Nothing'},
            {'id': 5,
            'name': 'Alex'}]

In [68]:
items = [1, 3, 5]


In [44]:
match

[1, 3]

In [38]:
for x in test_dict:
    print(x.get('id'))

1
2
None


In [49]:
tlist = [(d.get('id', 'default') for d in test_dict)]

In [50]:
tlist

[<generator object <genexpr> at 0x000002886D9990B0>]

In [54]:
match=[]
for i in items:
    for d in test_dict:
        if i in d.get('id', -9999):
            match.extend(i)

TypeError: argument of type 'int' is not iterable

In [69]:
match=[]
for i in items:
    for d in test_dict:
        if i == d.get('id', -9999):
            match.append(i)

In [70]:
match

[1, 5]

In [73]:
dict_list = [x for x in ((d.get('id', -9999) for d in test_dict))]
dict_list

[1, 2, -9999, 5]